In [1]:
def read_data(filename, encoding='cp949'): 
    with open(filename, 'r', encoding=encoding) as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] 
    return data
    
def write_data(data, filename, encoding='cp949'): 
    with open(filename, 'w', encoding=encoding) as f:
        f.write(data)

In [2]:
import pandas as pd

path = '/Users/jsha/gjai/nlp/'

data = pd.read_csv(path+'p_morphed.csv')
data = data[['연령', '감정_대분류', 'morphed']]
data

,연령,감정_대분류,morphed
0,청년,기쁨,아내 드디어 출산 되다 정말 신 나다 아 지금 정말 신 나다 아기 점점 크다 벌써 ...
1,노년,불안,당뇨 합병증 먹다 하다 약이 가지 넘어가다 스트레스 건강 때 관리 좀 잘 하다 하다...
2,청소년,당황,고등학교 올라오다 중학교 때다 수업 갑자기 어렵다 지다 당황 아직 학기 초인 나 수...
3,노년,기쁨,재취업 되다 받다 되다 첫 월급 오다 가족 외식 하다 예정 너무 행복 퇴직 후 다시...
4,노년,기쁨,빚 드디어 다 갚다 되다 이제야 안도감 들다 빚다 다 갚다 당분간 아무 생각 안 하...
...,...,...,...
40874,청년,불안,같이 살다 친구 애완견 데리다 오다 대부분 나 돌보다 있다 나 주인 혼란 나 아예 ...
40875,중년,기쁨,지난주 건강검진 결과 나오다 정상 결과 나오다 결과 좋다 나오다 다행
40876,청소년,슬픔,엄마 나 꿈 작가 응원 주다 하다 지금 안 그래 너무 슬프다 나 재능 남 월등 하다...
40877,노년,기쁨,이렇게 좋다 운동 시설 경로 우대 운동 있다 참 행운 덕분 건강 챙기다 있다 너무 ...


In [3]:
data = data.replace({'감정_대분류' : '불안 '}, '불안')     
data = data.replace({'감정_대분류' : '기쁨 '}, '기쁨') 

In [4]:
data['감정_대분류'].value_counts()

불안    7329
분노    6920
슬픔    6915
기쁨    6731
상처    6624
당황    6360
Name: 감정_대분류, dtype: int64

In [5]:
morphed_data = []
for i in data.index:
    each_data = data.iloc[i].to_list()
    each_data = list(data.iloc[i])
    morphed_data.append(each_data)

In [6]:
data_text = [line[2] for line in morphed_data] 
data_age = [line[0] for line in morphed_data] # 연령
data_senti = [line[1] for line in morphed_data] # 감정

In [7]:
# 감정
from sklearn.model_selection import train_test_split
train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, test_size=0.3, random_state=1111, stratify=data_senti)

In [8]:
# 감정
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)

test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq:', test_data_senti_freq)

train_data_senti_freq: Counter({'불안': 5130, '분노': 4844, '슬픔': 4840, '기쁨': 4712, '상처': 4637, '당황': 4452})
test_data_senti_freq: Counter({'불안': 2199, '분노': 2076, '슬픔': 2075, '기쁨': 2019, '상처': 1987, '당황': 1908})


In [9]:
import numpy as np
text_len = [len(line.split(' ')) for line in train_data_text] 
print("최소길이: ", np.min(text_len)) 
print("최대길이: ", np.max(text_len)) 
print("평균길이: ", np.round(np.mean(text_len), 1)) 
print("중위수길이: ", np.median(text_len)) 
print("구간별 최대 길이: ", np.percentile(text_len, [0, 25, 50, 75, 90, 99, 100]))

print("최소길이 문장: ", train_data_text[np.argmin(text_len)])
print("최대길이 문장: ", train_data_text[np.argmax(text_len)])

최소길이:  5
최대길이:  74
평균길이:  25.2
중위수길이:  25.0
구간별 최대 길이:  [ 5. 20. 25. 30. 35. 44. 74.]
최소길이 문장:  따돌림 힘들다 친구 나 따돌리다
최대길이 문장:  집안 크다 어른 부양 아래 독립 못 하다 약관 아들 있다 저 사업 망하다 대학 강의 코로나 이후 그것 못 하다 되다 경제적 매우 힘들다 그러게 예전 우리 나이 퇴직 연금 받다 느긋 사다 나인 우리 베이비 부 뭐 아직 일자리 수입 연연 팍팍 하다 생활 잇다 가다 하다 모으다 두다 돈 없다 그나마 건강 나쁘다 지다 않다 계약직 단순 아르바이트 가리다 않다 서너 동시 하다 오다 코로나 일자리 잡기 어렵다 지다 매일 위기


In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import math

max_words = 10000 
maxlen = 45

# 단어와 인덱스를 맵핑 (기준:빈도수)
# 단어를 기호로 바꾸는 토크나이저의 기준표는 일정해야 한다.
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data_text) 
word_index = tokenizer.word_index

Using TensorFlow backend.


In [11]:
print('전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index))
print('word_index type: ', type(word_index))
print('word_index: ', word_index)

전체에서 12645개의 고유한 토큰을 찾았습니다.
word_index type:  <class 'dict'>
word_index:  {'하다': 1, '나': 2, '보다': 3, '있다': 4, '친구': 5, '같다': 6, '되다': 7, '너무': 8, '없다': 9, '좋다': 10, '않다': 11, '주다': 12, '안': 13, '싶다': 14, '가다': 15, '사람': 16, '지다': 17, '받다': 18, '더': 19, '모르다': 20, '말': 21, '일': 22, '잘': 23, '생각': 24, '이야기': 25, '나다': 26, '들다': 27, '돈': 28, '정말': 29, '걱정': 30, '마음': 31, '힘들다': 32, '많이': 33, '회사': 34, '때': 35, '다': 36, '오다': 37, '아내': 38, '응': 39, '못': 40, '그': 41, '알다': 42, '이제': 43, '어떻하다': 44, '남편': 45, '오늘': 46, '기분': 47, '먹다': 48, '건강': 49, '못하다': 50, '요즘': 51, '많다': 52, '좀': 53, '지금': 54, '우리': 55, '다른': 56, '결혼': 57, '이번': 58, '집': 59, '살다': 60, '화': 61, '부모': 62, '가족': 63, '준비': 64, '열심히': 65, '아프다': 66, '맞다': 67, '엄마': 68, '자식': 69, '아들': 70, '나오다': 71, '슬프다': 72, '만나다': 73, '앞': 74, '왜': 75, '말다': 76, '내다': 77, '나이': 78, '시간': 79, '공부': 80, '혼자': 81, '다니다': 82, '이': 83, '직장': 84, '아이': 85, '생각하다': 86, '같이': 87, '대하다': 88, '학교': 89, '불안': 90, '대화': 91, '아직': 92, '고민': 93, '남자': 9

In [12]:
data = tokenizer.texts_to_sequences(train_data_text)
data = pad_sequences(data, maxlen=maxlen)
data

array([[   0,    0,    0, ...,  560,    8,  272],
       [   0,    0,    0, ...,  213,    4,    1],
       [   0,    0,    0, ...,   10,    3,   10],
       ...,
       [   0,    0,    0, ...,  300,   11,    6],
       [   0,    0,    0, ..., 1633,   12,   21],
       [   0,    0,    0, ...,   22,    9,   10]], dtype=int32)

In [13]:
def to_one_hot(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = to_one_hot(data, dimension=max_words)

In [14]:
# 감정
def sentiToIint(sentis):
    for i, senti in enumerate(sentis):
        if senti == "불안":
            sentis[i] = 0
        elif senti == "분노":
            sentis[i] = 1
        elif senti == "슬픔":
            sentis[i] = 2
        elif senti == "기쁨":
            sentis[i] = 3
        elif senti == "상처":
            sentis[i] = 4
        elif senti == "당황":
            sentis[i] = 5
    return sentis

In [15]:
# 감정
train_data_senti = sentiToIint(train_data_senti)
test_data_senti = sentiToIint(test_data_senti)

In [16]:
class_number = 6
y_train = to_one_hot(train_data_senti, dimension=class_number)
y_train

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [17]:
validation_ratio = 0.3 
validation_len = math.floor(len(train_data_text) * validation_ratio)

x_train = x_train[validation_len:] 
y_train = y_train[validation_len:] 
x_val = x_train[:validation_len] 
y_val = y_train[:validation_len]

In [18]:
# 감정
data_test = tokenizer.texts_to_sequences(test_data_text)
data_test = pad_sequences(data_test, maxlen=maxlen)
x_test = to_one_hot(data_test, dimension=max_words)
y_test = to_one_hot(test_data_senti, dimension=class_number)

In [19]:
x_test = x_test[:3000]
y_test = y_test[:3000]

In [20]:
print('훈련 데이터 본문 텐서의 크기:', x_train.shape)
print('훈련 데이터 레이블 텐서의 크기:', y_train.shape)

print('테스트 데이터 텐서의 크기:', x_test.shape)
print('테스트 레이블 텐서의 크기:', y_test.shape)

훈련 데이터 본문 텐서의 크기: (20031, 10000)
훈련 데이터 레이블 텐서의 크기: (20031, 6)
테스트 데이터 텐서의 크기: (3000, 10000)
테스트 레이블 텐서의 크기: (3000, 6)


In [21]:
epochs = 20 
batch_size = 50 
model_name = 'train_data_morphed2.h5'
tokenizer_name = 'train_data_morphed2.pickle'

In [22]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential() 

model.add(Dense(128, activation='relu', input_shape=(max_words,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(units=32, activation='relu')) 
model.add(Dense(units=class_number, activation='softmax')) 

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1280128   
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 1,290,662
Trainable params: 1,290,662
Non-trainable params: 0
_________________________________________________________________


In [24]:
from keras.callbacks import EarlyStopping
import os

checkpoint_path = path+'jaeeun/'
# checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_path):
    print("{} -- Folder already exists\n".format(checkpoint_path))
else:
    os.makedirs(checkpoint_path, exist_ok=True)
    print("{} -- Folder created\n".format(checkpoint_path))

/Users/jsha/gjai/nlp/jaeeun/ -- Folder already exists



In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [26]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))
history_dict = history.history

Train on 20031 samples, validate on 8584 samples
Epoch 1/20
20031/20031 [==============================] - 10s 476us/step - loss: 1.1797 - acc: 0.5689 - val_loss: 0.6655 - val_acc: 0.7877
Epoch 2/20
20031/20031 [==============================] - 10s 479us/step - loss: 0.7103 - acc: 0.7587 - val_loss: 0.4246 - val_acc: 0.8772
Epoch 3/20
20031/20031 [==============================] - 9s 443us/step - loss: 0.4562 - acc: 0.8482 - val_loss: 0.2363 - val_acc: 0.9380
Epoch 4/20
20031/20031 [==============================] - 9s 441us/step - loss: 0.2458 - acc: 0.9262 - val_loss: 0.0959 - val_acc: 0.9787
Epoch 5/20
20031/20031 [==============================] - 9s 441us/step - loss: 0.1023 - acc: 0.9716 - val_loss: 0.0417 - val_acc: 0.9913
Epoch 6/20
20031/20031 [==============================] - 9s 455us/step - loss: 0.0405 - acc: 0.9907 - val_loss: 0.0175 - val_acc: 0.9972
Epoch 7/20
20031/20031 [==============================] - 10s 478us/step - loss: 0.0146 - acc: 0.9972 - val_loss: 0.0086 

In [27]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

print('Validation accuracy of each epoch:', np.round(val_acc, 3))
epochs = range(1, len(val_acc) + 1)

Validation accuracy of each epoch: [0.788 0.877 0.938 0.979 0.991 0.997 0.999 1.    0.999 1.    1.    1.
 1.    0.984 0.988 0.999 1.    1.    1.    1.   ]


In [28]:
import pickle
with open(path+tokenizer_name, 'wb') as file:
    pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
model.save(path+model_name)

In [30]:
test_eval = model.evaluate(x_test, y_test)
test_eval

3000/3000 [==============================] - 0s 134us/step


[3.4553789529800416, 0.6156666874885559]

In [34]:
def intToSenti(senti_int):
    sentis = ''
    if senti_int == 0:
        sentis = '불안'
    elif senti_int == 1:
        sentis = '분노'
    elif senti_int == 2:
        sentis = '슬픔'
    elif senti_int == 3:
        sentis = '기쁨'
    elif senti_int == 4:
        sentis = '상처'
    elif senti_int == 5:
        sentis = '당황'
    return sentis